# Py file testing

In [666]:
from ACF_GNR_estimation_functions import *

In [668]:
df_ACF = load_data("ACF")
df_GNR = load_data("GNR")

In [670]:
theta0 = np.array([1,1])/2
coeffs_ACF, convergence = ACF_estimation(df_ACF, theta0, print_results=1)
theta_ACF = coeffs_ACF[1:3]

The gradient at the optimum is:  [1.19064544e-14 7.09991245e-15]
The GMM error using the gradient is: 1.3973985035997367e-28
The estimates using autograd: [beta_k, beta_l] =  [0.32356761 0.73189068]
The slope of the AR(1) of productivity is: rho =  0.8821586747079723


### Note: when bootstrapping, the rho falls. Why? It has to do with the time series properties of the data. 
### Use "block bootstrap" designed for panel data. 
This samples the firms with replacement. 

In [672]:
#n_firm_samples = len(df_ACF['firm_id'].unique())
df_boot_ACF = bootstrap_sample_panel(df_ACF, 1)
dfb1 = df_boot_ACF[0]
coeffs_ACF_boot, _ = ACF_estimation(dfb1, theta_ACF, print_results=1)

The gradient at the optimum is:  [-1.24495888e-14 -7.97498356e-15]
The GMM error using the gradient is: 1.7469718081992206e-28
The estimates using autograd: [beta_k, beta_l] =  [0.29802329 0.7807325 ]
The slope of the AR(1) of productivity is: rho =  0.8844675799404267


In [674]:
alpha0 = np.ones(5)/2 #This is the required size to have coefficeints for k, l, kl, k**2, l**2. Need to change if the degree is changed
gammaprime0 = np.ones(10)/2  #Also needs to change if the degree is changed

initial_guesses0 = (alpha0, gammaprime0)

results_params_GNR, results_convergence_GNR, alpha_GNR, gammaprime_GNR = GNR_estimation(df_GNR, initial_guesses0, print_results = 1)

initial_guesses_GNR = (alpha_GNR, gammaprime_GNR)

The error is: 2.017527216637981e-24
The gradient is: [5.73400746e-11 3.35555990e-11 1.37255468e-09 7.02585598e-10
 3.46728579e-10]
The coefficients for the integration constant [alpha] are: [ 0.22962852 -2.6976986  -0.08259352  0.26112497  0.05193076]
The coefficients for productivity omega [delta] are: [1.97089299 0.17895714 0.08014284]
the average productivity [omega] is: 3.8396919941227203
the average elasticity [df/dm] is: 0.569660785277684
----Assuming Cobb-Douglas----
[beta_0, beta_k, beta_l, beta_m] =  [-0.21136415  0.19574309  0.31153059  0.46819569]


In [676]:
df_boot_GNR = bootstrap_sample_panel(df_GNR, 1)
dfb1GNR = df_boot_GNR[0]
_, _, _, _ = GNR_estimation(dfb1GNR, initial_guesses_GNR , print_results = 1)

The error is: 2.0990131401017668e-24
The gradient is: [2.99701564e-11 1.70933783e-11 7.26522949e-10 3.59416382e-10
 1.70206466e-10]
The coefficients for the integration constant [alpha] are: [ 1.24800574 -5.46332764 -0.18720228  0.64786964 -0.12578311]
The coefficients for productivity omega [delta] are: [1.22047973 0.71622548 0.01033014]
the average productivity [omega] is: 5.287103084969627
the average elasticity [df/dm] is: 0.5929509691528064
----Assuming Cobb-Douglas----
[beta_0, beta_k, beta_l, beta_m] =  [-1.65982147  0.13552359  0.35405632  0.5051003 ]


# Bootstrap:

In [678]:
n_boot_samples = 10

In [699]:
#Use the initial condition from the true data, coeffs_ACF[:2], to improve speed and convergence. 
bootstrap_results_ACF, convergence_ACF = bootstrap(ACF_estimation, theta_ACF, df_ACF, n_boot_samples)

#Summarize array
ACF_row_names = np.array(["beta_0", "beta_k", "beta_l", "rho", "Eomega", "gmm_error"])
ACF_summary = summarize_array(coeffs_ACF, bootstrap_results_ACF, ACF_row_names[:-1])

boot_full_ACF = pd.DataFrame(np.hstack((bootstrap_results_ACF, convergence_ACF)), columns = ACF_row_names) 

#Save to CSV
ACF_summary.to_csv("../Results/summary_stats_ACF.csv")
boot_full_ACF.to_csv("../Results/full_bootstrap_ACF.csv")

In [709]:
#Use the initial condition from the true data, coeffs_ACF[:2], to improve speed and convergence. 
bootstrap_results_GNR, convergence_GNR = bootstrap(GNR_estimation, initial_guesses_GNR, df_GNR, n_boot_samples, columns = 6)

#Summarize array
GNR_row_names = np.array(["beta_0_cd", "beta_k_cd", "beta_l_cd", "beta_m_cd", "Edf_dm", "Eomega", "gmm_error"])
GNR_summary = summarize_array(results_params_GNR, bootstrap_results_GNR, GNR_row_names[:-1])

boot_full_GNR = pd.DataFrame(np.hstack((bootstrap_results_GNR, convergence_GNR)), columns = GNR_row_names) 


#Save to CSV
GNR_summary.to_csv("../Results/summary_stats_GNR.csv")
boot_full_GNR.to_csv("../Results/full_bootstrap_GNR.csv")

convergence failed on sample 6 ; trying new initial guess.
convergence succeeded on sample 6
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence failed on sample 7 ; trying new initial guess.
convergence succeeded on sample 7
convergence failed on sample 8 ; trying new initial guess.
convergence succeeded on sample 8
convergence failed on sample 9 ; trying new initial guess.
convergence succeeded on sample 9
